In [8]:
import torch
import torch.nn as nn
import numpy as np

dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(dev)


cuda


In [31]:
emojis = {
    'hat': '\U0001F3A9',
    'rat': '\U0001F400',
    'cat': '\U0001F408',
    'flat': '\U0001F3E2',
    'matt': '\U0001F468',
    'cap': '\U0001F9E2',
    'son': '\U0001F466'
}

index_to_emoji = [value for _, value in emojis.items()]
emojies = np.eye(len(emojis))
emoji_encoding_size = len(emojies)

index_to_char = [' ', 's', 'a', 'r', 'm', 'p', 'c', 't', 'f', 'h', 'l', 'n', 'o']
char_encodings = np.eye(len(index_to_char))
encoding_size = len(char_encodings)

letters ={}

for i, letter in enumerate(index_to_char):
        letters[letter] = char_encodings[i]

# Generate list of index to char
# words = ("hat ", "rat ", "cat ", "flat ", "matt ", "cap ", "son ")
# letters = list(map(lambda w: list(w), words))
# import functools
# print(set(functools.reduce(lambda a, b: a + b, words)))

x_train = torch.tensor([
        [[letters['h']], [letters['a']], [letters['t']], [letters[' ']]],
        [[letters['r']], [letters['a']], [letters['t']], [letters[' ']]],
        [[letters['c']], [letters['a']], [letters['t']], [letters[' ']]],
        [[letters['f']], [letters['l']], [letters['a']], [letters['t']]],
        [[letters['m']], [letters['a']], [letters['t']], [letters['t']]],
        [[letters['c']], [letters['a']], [letters['p']], [letters[' ']]],
        [[letters['s']], [letters['o']], [letters['n']], [letters[' ']]],
        ], 
        dtype=torch.float)

y_train = torch.tensor([
        [emojies[0], emojies[0], emojies[0], emojies[0]] ,
        [emojies[1], emojies[1], emojies[1], emojies[1]],
        [emojies[2], emojies[2], emojies[2], emojies[2]],
        [emojies[3], emojies[3], emojies[3], emojies[3]],
        [emojies[4], emojies[4], emojies[4], emojies[4]],
        [emojies[5], emojies[5], emojies[5], emojies[5]],
        [emojies[6], emojies[6], emojies[6], emojies[6]]], 
        dtype=torch.float)

torch.Size([7, 4, 1, 13])
torch.Size([7, 4, 7])


In [36]:
class LongShortTermMemoryModel(nn.Module):
    def __init__(self, encoding_size, emoji_encoding_size):
        super(LongShortTermMemoryModel, self).__init__()
        self.lstm = nn.LSTM(encoding_size, 128)  # 128 is the state size
        self.dense = nn.Linear(128, emoji_encoding_size)  # 128 is the state size

    def reset(self):  # Reset states prior to new input sequence
        zero_state = torch.zeros(1, 1, 128).to(dev)  # Shape: (number of layers, batch size, state size)
        self.hidden_state = zero_state
        self.cell_state = zero_state

    def logits(self, x):  # x shape: (sequence length, batch size, encoding size)
        out, (self.hidden_state, self.cell_state) = self.lstm(x, (self.hidden_state, self.cell_state))
        return self.dense(out.reshape(-1, 128))

    def f(self, x):  # x shape: (sequence length, batch size, encoding size)
        return torch.softmax(self.logits(x), dim=1)

    def loss(self, x, y):  # x shape: (sequence length, batch size, encoding size), y shape: (sequence length, encoding size)
        return nn.functional.cross_entropy(self.logits(x), y.argmax(1))

In [38]:
model = LongShortTermMemoryModel(encoding_size, emoji_encoding_size).to(dev)

optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
for epoch in range(500):
    for i in range(x_train.size()[0]):
        model.reset()
        model.loss(x_train[i].to(dev), y_train[i].to(dev)).backward()
        optimizer.step()
        optimizer.zero_grad()

In [42]:
def generate_emoji(string):
    y = -1
    model.reset()
    for i in range(len(string)):
        char_index = index_to_char.index(string[i])
        y = model.f(torch.tensor([[char_encodings[char_index]]], dtype=torch.float).to(dev))
    print(index_to_emoji[y.argmax(1)])

generate_emoji('rt')
generate_emoji('rats')
generate_emoji("s")

🐀
🐀
👦
